<a href="https://colab.research.google.com/github/eyaguirat10/CoWin-Breast-Cancer-Detection/blob/main/Breast_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Business Understanding

# Breast Cancer Diagnosis Project

## **Summary**
The project aims to improve breast cancer diagnosis by automating the classification of tumors (benign or malignant) using machine learning algorithms on the Wisconsin Diagnostic Breast Cancer (WDBC) dataset.

---

## **Stakeholders**
- **Oncologists, radiologists, pathologists:** Primary users of predictions.  
- **Hospital IT/data teams:** Responsible for integration, security, and maintenance.  
- **Patients:** Indirect beneficiaries via improved diagnostics.  

---

## **Supervised Learning**

**Business Objective (BO):**  
Develop a reliable, accurate system for breast cancer detection.

**Data Science Objective (DSO):**  
Predict the diagnosis type — **M (Malignant)** or **B (Benign)**.

**Business Success Metrics**  
These metrics require true labels to evaluate prediction performance:  
- **Accuracy:** Overall proportion of correct classifications. **Target ≥ 98%**  
- **Precision:** Reliability of positive (malignant) predictions. **Target ≥ 97%**  
- **F1-Score:** Balances recall and precision, especially with class imbalance. **Target ≥ 97%**  
- **AUC-ROC:** Measures ability to discriminate between classes. **Target ≥ 0.99**

---

## **Unsupervised Learning**

**Business Objective (BO):**  
Identify and group two types of breast cancer cases.

**Data Science Objective (DSO):**  
Cluster and classify diagnosis patterns without predefined labels.

**Business Success Metrics**  
Since labels are not available, success is measured differently:  
- **Silhouette Score:** Measures how well-separated clusters are.  
- **Davies-Bouldin Index / Calinski-Harabasz Index:** Assess cluster compactness and separation.  
- **Domain Interpretability:** Clusters should be medically meaningful (e.g., malignant vs benign enrichment).


#III. Data Preparation

#IV. Modeling

#V. Evaluation

#VI. Deployment